## 스타벅스 매장 데이터 크롤링 & 메가커피 크롤링데이터 병합 전처리

# < 스타벅스 매장 데이터 크롤링 >

## 필요 라이브러리 추가

In [235]:
import requests
from urllib import parse
import pandas as pd
import numpy as np
import seaborn as sns
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import os
import time

## 지도 형식으로 데이터가 존재한 것이 확인되어 크롬 드라이버 형식으로 데이터 추출시도

In [236]:
browser = webdriver.Chrome()
url = 'https://www.starbucks.co.kr/store/store_map.do?disp=locale'
browser.get(url)

In [237]:
# 서울부터 세종까지(1~17번) 지역별 구분하여 데이터를 저장한 형태를 띄고 있어 for문 활용 조회
for i in range(1,18):
    region_btn ='#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) \t
    > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child({}) > a'.format(i) #li:nth-child(1~17)형식임을 발견했으므로 번호대로 조회
    browser.find_element_by_css_selector(region_btn).click()
    time.sleep(1)
    try: # 지역구가 없는 세종시 존재 -> 지역구가 있는 경우에만 사용을 위해 try 사용
        all_btn = '#mCSB_2_container > ul > li:nth-child(1) > a'
        browser.find_element_by_css_selector(all_btn).click() # 지역구 전체 클릭
        time.sleep(4) # 데이터로딩 시간 고려해서 할당
        html = browser.page_source # 크롬 브라우저에 현재 화면에 나타난 웹페이지의 html을 가져옴
        soup = BeautifulSoup(html,'html.parser')
        starbucks_soup_list = soup.select('li.quickResultLstCon')

        starbucks_list =[]
        # 스타벅스의 경우, 데이터 구조값 내에 주소, 위도,경도 값이 있어 해당값 추출형식 활용
        for item in starbucks_soup_list:
            name = item.select('strong')[0].text.strip()
            address = str(item.select('p.result_details')[0]).split('<br/')[0].split('>')[1]
            lat= item['data-lat'].strip()
            lng= item['data-long'].strip()

            starbucks_list.append([name,address,lat,lng])

        columns =['매장명','주소','위도','경도']
        seoul_starbucks_df = pd.DataFrame(starbucks_list,columns=columns)
        
        # 지역별 분석을 위해 주소 분할
        지역 = []
        군구 = []
        세부 = []

        for e in seoul_starbucks_df['주소']:
            지역.append(e.split(' ')[0])
            군구.append(e.split(' ')[1])
            세부.append(e.split(' ')[2])

            지역_np = np.array(지역)
            군구_np = np.array(군구)
            세부_np = np.array(세부)

            지역_df = pd.DataFrame(지역_np)
            군구_df = pd.DataFrame(군구_np)
            세부_df = pd.DataFrame(세부_np)

            merge_df = pd.concat([지역_df,군구_df,세부_df],axis=1)
            newdf = pd.concat([seoul_starbucks_df,merge_df],axis=1)
        newdf.columns = ['매장명','주소','위도','경도','지역','시군구','세부']
        newdf.to_csv('./1차프로젝트 데이터/starbucks_store_list{}.csv'.format(i))
        # 저장 완료 후 지역검색 클릭 -> 전체 조회가 다시 가능하게 만듬 
        back_btn ='#container > div > form > fieldset > div > section > article.find_store_cont > article > header.loca_search > h3 > a'
        browser.find_element_by_css_selector(back_btn).click()
        time.sleep(1)
        
    except: # 세종시 같은 예외적인 지역을 고려한 위한 복사.붙여넣기
        html = browser.page_source # 크롬 브라우저에 현재 화면에 나타난 웹페이지의 html을 가져옴
        soup = BeautifulSoup(html,'html.parser')
        starbucks_soup_list = soup.select('li.quickResultLstCon')

        starbucks_list =[]
        for item in starbucks_soup_list:
            name = item.select('strong')[0].text.strip()
            address = str(item.select('p.result_details')[0]).split('<br/')[0].split('>')[1]
            lat= item['data-lat'].strip()
            lng= item['data-long'].strip()

            starbucks_list.append([name,address,lat,lng])

        columns =['매장명','주소','위도','경도']
        seoul_starbucks_df = pd.DataFrame(starbucks_list,columns=columns)

        지역 = []
        군구 = []
        세부 = []

        for e in seoul_starbucks_df['주소']:
            지역.append(e.split(' ')[0])
            군구.append(e.split(' ')[1])
            세부.append(e.split(' ')[2])

            지역_np = np.array(지역)
            군구_np = np.array(군구)
            세부_np = np.array(세부)

            지역_df = pd.DataFrame(지역_np)
            군구_df = pd.DataFrame(군구_np)
            세부_df = pd.DataFrame(세부_np)

            merge_df = pd.concat([지역_df,군구_df,세부_df],axis=1)
            newdf = pd.concat([seoul_starbucks_df,merge_df],axis=1)
        newdf.columns = ['매장명','주소','위도','경도','지역','시군구','세부']
        newdf.to_csv('./1차프로젝트 데이터/starbucks_store_list{}.csv'.format(i))
        
        back_btn ='#container > div > form > fieldset > div > section > article.find_store_cont > article > header.loca_search > h3 > a'
        browser.find_element_by_css_selector(back_btn).click()
        time.sleep(1)


In [238]:
# 저장 데이터 확인
pd.read_csv('./1차프로젝트 데이터/starbucks_store_list17.csv')

Unnamed: 0        매장명                                                 주소  \
0            0     세종다정DT                             세종특별자치시 다정북로 219 (다정동)   
1            1       세종보람  세종특별자치시 한누리대로 2165 (보람동) A118,119,120,121,128,...   
2            2       세종새롬                            세종특별자치시  새롬중앙로 64 (새롬동)   
3            3      세종어진R                           세종특별자치시  한누리대로 499 (어진동)   
4            4       세종종촌                   세종특별자치시  도움3로 105-10 (종촌동) 성운프라자   
5            5      세종첫마을                         세종특별자치시  한누리대로 169 행복타워 1층   
6            6      세종이마트                   세종특별자치시 금송로 687(가람동 406) 이마트 세종점   
7            7       세종청사                             세종특별자치시  도움8로 81 (어진동)   
8            8       세종아름      세종특별자치시 아름서1길 13-9 (아름동) 103,104,105,202,203호   
9            9       세종소담  세종특별자치시 소담3로 8 (소담동) 1동  12호,13호,14호,15호,15A호,16호   
10          10  세종충남대학교병원                    세종특별자치시 보듬7로 20 (도담동) 헬스케어센터 1층   

           위도          경도       지역    시군구       세부  
0   36.494878  127.255384  세종특별자치시   다정북로      219  
1   36.477332  127.286519  세종특별자치시  한누리대로     2165  
2   36.486896  127.250718  세종특별자치시    NaN    새롬중앙로  
3   36.508188  127.261188  세종특별자치시    NaN    한누리대로  
4   36.504112  127.249381  세종특별자치시    NaN     도움3로  
5   36.480383  127.261795  세종특별자치시    NaN    한누리대로  
6   36.471219  127.250997  세종특별자치시    금송로  687(가람동  
7   36.504565  127.270769  세종특별자치시    NaN     도움8로  
8   36.511880  127.242770  세종특별자치시  아름서1길     13-9  
9   36.485755  127.300509  세종특별자치시   소담3로        8  
10  36.519360  127.258280  세종특별자치시   보듬7로       20

## 분할 데이터 병합

In [256]:
# 분할 데이터 병합 -> 동일한 방법으로 페이지별 분할 저장되어 있는 메가커피 데이터도 병합 후 저장
import glob

input_file = r'C:\Users\dmsck\Desktop\K-digital 팀프로젝트\1차 프로젝트\1차프로젝트 데이터\스타벅스' # csv파일들이 있는 디렉토리 위치
output_file = r'C:\Users\dmsck\Desktop\K-digital 팀프로젝트\1차 프로젝트\1차프로젝트 데이터\result(스타벅스).csv' # 병합하고 저장하려는 파일명

allFile_list = glob.glob(os.path.join(input_file, 'starbucks_*')) # glob함수로 sales_로 시작하는 파일들을 모음
print(allFile_list)
allData = [] # 읽어 들인 csv파일 내용을 저장할 빈 리스트
for file in allFile_list:
    df = pd.read_csv(file) # for구문으로 csv파일들을 읽어 들인다
    allData.append(df) # 빈 리스트에 읽어 들인 내용을 추가한다

dataCombine = pd.concat(allData, axis=0, ignore_index=True) # concat함수를 이용해서 리스트의 내용을 병합
# axis=0은 수직으로 병합함. axis=1은 수평. ignore_index=True는 인데스 값이 기존 순서를 무시하고 순서대로 정렬되도록 함.
dataCombine.to_csv(output_file, index=False) # to_csv함수로 저장. 인데스를 빼려면 False로 설정

['C:\\Users\\dmsck\\Desktop\\K-digital 팀프로젝트\\1차 프로젝트\\1차프로젝트 데이터\\스타벅스\\starbucks_store_list1.csv', 'C:\\Users\\dmsck\\Desktop\\K-digital 팀프로젝트\\1차 프로젝트\\1차프로젝트 데이터\\스타벅스\\starbucks_store_list10.csv', 'C:\\Users\\dmsck\\Desktop\\K-digital 팀프로젝트\\1차 프로젝트\\1차프로젝트 데이터\\스타벅스\\starbucks_store_list11.csv', 'C:\\Users\\dmsck\\Desktop\\K-digital 팀프로젝트\\1차 프로젝트\\1차프로젝트 데이터\\스타벅스\\starbucks_store_list12.csv', 'C:\\Users\\dmsck\\Desktop\\K-digital 팀프로젝트\\1차 프로젝트\\1차프로젝트 데이터\\스타벅스\\starbucks_store_list13.csv', 'C:\\Users\\dmsck\\Desktop\\K-digital 팀프로젝트\\1차 프로젝트\\1차프로젝트 데이터\\스타벅스\\starbucks_store_list14.csv', 'C:\\Users\\dmsck\\Desktop\\K-digital 팀프로젝트\\1차 프로젝트\\1차프로젝트 데이터\\스타벅스\\starbucks_store_list15.csv', 'C:\\Users\\dmsck\\Desktop\\K-digital 팀프로젝트\\1차 프로젝트\\1차프로젝트 데이터\\스타벅스\\starbucks_store_list16.csv', 'C:\\Users\\dmsck\\Desktop\\K-digital 팀프로젝트\\1차 프로젝트\\1차프로젝트 데이터\\스타벅스\\starbucks_store_list17.csv', 'C:\\Users\\dmsck\\Desktop\\K-digital 팀프로젝트\\1차 프로젝트\\1차프로젝트 데이터\\스타벅스\\starbucks_store_lis

# 스타벅스 주소 데이터 병합 전처리

In [260]:
# 주소데이터 불러오기
star_df = pd.read_csv('result(스타벅스).csv')

In [264]:
# 두 데이터 브랜드 구분을 위해 column 생성
star_df['Unnamed: 0'] = '스타벅스'
star_df

Unnamed: 0      매장명                          주소         위도          경도  \
0          스타벅스  역삼아레나빌딩     서울특별시 강남구 언주로 425 (역삼동)  37.501087  127.043069   
1          스타벅스   논현역사거리    서울특별시 강남구 강남대로 538 (논현동)  37.510178  127.022223   
2          스타벅스  신사역성일빌딩    서울특별시 강남구 강남대로 584 (논현동)  37.514132  127.020563   
3          스타벅스   국기원사거리    서울특별시 강남구 테헤란로 125 (역삼동)  37.499517  127.031495   
4          스타벅스   스탈릿대치R  서울특별시 강남구 남부순환로 2947 (대치동)  37.494668  127.062583   
...         ...      ...                         ...        ...         ...   
1604       스타벅스   원주반곡DT      강원도 원주시 동부순환로 37 (반곡동)  37.319904  127.971570   
1605       스타벅스   원주명륜DT       강원도 원주시 남원로 588 (명륜동)  37.335971  127.949754   
1606       스타벅스     원주무실       강원도 원주시 능라동길 73 (무실동)  37.332972  127.930448   
1607       스타벅스   원주혁신도시        강원도 원주시 혁신로 61 (반곡동)  37.323157  127.980611   
1608       스타벅스    원주터미널      강원도 원주시 서원대로 178 (단계동)  37.343887  127.931100   

         지역  시군구     세부  
0     서울특별시  강남구    언주로  
1     서울특별시  강남구   강남대로  
2     서울특별시  강남구   강남대로  
3     서울특별시  강남구   테헤란로  
4     서울특별시  강남구  남부순환로  
...     ...  ...    ...  
1604    강원도  원주시  동부순환로  
1605    강원도  원주시    남원로  
1606    강원도  원주시   능라동길  
1607    강원도  원주시    혁신로  
1608    강원도  원주시   서원대로  

[1609 rows x 8 columns]

In [271]:
# column명 지정
star_df1 = star_df.rename(columns={'Unnamed: 0':'브랜드'})
star_df1

브랜드      매장명                          주소         위도          경도     지역  \
0     스타벅스  역삼아레나빌딩     서울특별시 강남구 언주로 425 (역삼동)  37.501087  127.043069  서울특별시   
1     스타벅스   논현역사거리    서울특별시 강남구 강남대로 538 (논현동)  37.510178  127.022223  서울특별시   
2     스타벅스  신사역성일빌딩    서울특별시 강남구 강남대로 584 (논현동)  37.514132  127.020563  서울특별시   
3     스타벅스   국기원사거리    서울특별시 강남구 테헤란로 125 (역삼동)  37.499517  127.031495  서울특별시   
4     스타벅스   스탈릿대치R  서울특별시 강남구 남부순환로 2947 (대치동)  37.494668  127.062583  서울특별시   
...    ...      ...                         ...        ...         ...    ...   
1604  스타벅스   원주반곡DT      강원도 원주시 동부순환로 37 (반곡동)  37.319904  127.971570    강원도   
1605  스타벅스   원주명륜DT       강원도 원주시 남원로 588 (명륜동)  37.335971  127.949754    강원도   
1606  스타벅스     원주무실       강원도 원주시 능라동길 73 (무실동)  37.332972  127.930448    강원도   
1607  스타벅스   원주혁신도시        강원도 원주시 혁신로 61 (반곡동)  37.323157  127.980611    강원도   
1608  스타벅스    원주터미널      강원도 원주시 서원대로 178 (단계동)  37.343887  127.931100    강원도   

      시군구     세부  
0     강남구    언주로  
1     강남구   강남대로  
2     강남구   강남대로  
3     강남구   테헤란로  
4     강남구  남부순환로  
...   ...    ...  
1604  원주시  동부순환로  
1605  원주시    남원로  
1606  원주시   능라동길  
1607  원주시    혁신로  
1608  원주시   서원대로  

[1609 rows x 8 columns]

In [281]:
# 원하는 데이터 열 정리 후 저장 -> 브랜드 구분 데이터 확보
star_df1 = star_df1[['브랜드','매장명','주소','지역','시군구','세부','위도','경도']]
star_df1.to_csv('매장데이터전처리_스타벅스.csv')

# < 메가커피 >

# 메가커피 매장 크롤링 분할 데이터 병합 진행

In [269]:
mega_df = pd.read_csv('result.csv')
mega_df['Unnamed: 0'] = '메가커피'
mega_df1 = mega_df.rename(columns={'Unnamed: 0':'브랜드'})

In [280]:
mega_df1 = mega_df1[['브랜드','매장명','주소','지역','시군구','세부','위도','경도']]
mega_df1
mega_df1.to_csv('매장데이터전처리_메가커피.csv')

# 스타벅스와 메가커피 전처리 데이터 병합

In [278]:
# 두 브랜드 전처리 데이터 병합
최종 = pd.concat([star_df1,mega_df1])
최종.to_csv('매장데이터전처리.csv')

In [279]:
pd.read_csv('매장데이터전처리.csv')

Unnamed: 0   브랜드       매장명                          주소     지역   시군구  \
0              0  스타벅스   역삼아레나빌딩     서울특별시 강남구 언주로 425 (역삼동)  서울특별시   강남구   
1              1  스타벅스    논현역사거리    서울특별시 강남구 강남대로 538 (논현동)  서울특별시   강남구   
2              2  스타벅스   신사역성일빌딩    서울특별시 강남구 강남대로 584 (논현동)  서울특별시   강남구   
3              3  스타벅스    국기원사거리    서울특별시 강남구 테헤란로 125 (역삼동)  서울특별시   강남구   
4              4  스타벅스    스탈릿대치R  서울특별시 강남구 남부순환로 2947 (대치동)  서울특별시   강남구   
...          ...   ...       ...                         ...    ...   ...   
3136        1527  메가커피       삼양점    서울특별시 강북구 솔샘로 213-2(미아동)     서울   강북구   
3137        1528  메가커피     인천동양점      인천광역시 계양구 동양로 117(동양동)     인천   계양구   
3138        1529  메가커피     구로구청점     서울특별시 구로구 가마산로 268(구로동)     서울   구로구   
3139        1530  메가커피  의정부민락KT점       경기도 의정부시 용현로 154(민락동)     경기  의정부시   
3140        1531  메가커피      간석역점       인천광역시 남동구 석산로 15(간석동)     인천   남동구   

         세부         위도          경도  
0       언주로  37.501087  127.043069  
1      강남대로  37.510178  127.022223  
2      강남대로  37.514132  127.020563  
3      테헤란로  37.499517  127.031495  
4     남부순환로  37.494668  127.062583  
...     ...        ...         ...  
3136    미아동  37.620216  127.016487  
3137    동양동  37.559109  126.757883  
3138    구로동  37.495616  126.890319  
3139    민락동  37.740523  127.093060  
3140    간석동  37.463633  126.695163  

[3141 rows x 9 columns]